In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pandas as pd
from datetime import datetime

In [2]:
import pyarrow.parquet as pq

In [3]:
df = pq.ParquetFile("./data/pq/green/2019/01/part-00000-a9762b67-85ff-4157-8b2c-ec9814daf9a4-c000.snappy.parquet")

In [ ]:
table_parque = df.num_row_groups
print(table_parque)

In [ ]:
# for row in range(table_parque):
#     x = df.read_row_groups(row,use_pandas_metadata=True).to_pandas()
    

In [4]:
conf = SparkConf()
conf.setMaster("local[*]").setAppName("into_rdd")
sc = SparkContext.getOrCreate(conf)
sc.defaultParallelism
spark = SQLContext(sc)
spark.setConf("spark.default.parallelism", "100")
sc._conf.getAll()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 12:48:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/20 12:48:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/usr/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.name', 'into_rdd'),
 ('spark.app.startTime', '1676872093757'),
 ('spark.driver.port', '32921'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', 'sapautraArch'),
 ('spark.rdd.compress', 'True'),
 ('spark.executor

In [178]:
sc.stop()

```
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 

    -- Revenue sum
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
-- ORDER BY 
--     1, 2
```

In [5]:
df_green = spark.read.parquet('./data/pq/green/*/*')

In [6]:
df_yellow = spark.read.parquet('./data/pq/yellow/*/*')

In [15]:
df_yellow.head(5)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2019, 3, 1, 21, 17, 13), tpep_dropoff_datetime=datetime.datetime(2019, 3, 1, 21, 23, 3), passenger_count=1, trip_distance=0.68, RatecodeID=1, store_and_fwd_flag='N', PULocationID=114, DOLocationID=79, payment_type=1, fare_amount=5.5, extra=0.5, mta_tax=0.5, tip_amount=1.86, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=11.16, congestion_surcharge=2.5),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2019, 3, 14, 8, 30, 52), tpep_dropoff_datetime=datetime.datetime(2019, 3, 14, 8, 36, 11), passenger_count=1, trip_distance=0.84, RatecodeID=1, store_and_fwd_flag='N', PULocationID=48, DOLocationID=161, payment_type=1, fare_amount=5.5, extra=0.0, mta_tax=0.5, tip_amount=1.2, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=10.0, congestion_surcharge=2.5),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2019, 3, 16, 17, 51, 8), tpep_dropoff_datetime=datetime.datetime(2019, 3, 16, 17, 59, 55), passenge

In [4]:
rdd = df_yellow.select('tpep_pickup_datetime','PULocationID','total_amount').rdd

NameError: name 'df_yellow' is not defined

# Where Clause in rdd

In [ ]:
rdd.filter(lambda row: True).take(1)

In [ ]:
start = datetime(2021,1,1)

In [ ]:
def filter_outliers(row):
    return row.tpep_pickup_datetime >= start


In [ ]:
rdd.filter(filter_outliers).take(1)

In [ ]:
rows = rdd.take(10)
row = rows[0]

In [ ]:
row

# Group BY

#### Prepare Grouping is create new rdd with structure data is (key,value)

In [ ]:
def prepareForGrouping(row):
    hour = row.tpep_pickup_datetime.replace(minute=0,second=0,microsecond=0)
    zone = row.PULocationID

    count = 1
    amount = row.total_amount
    
    key = (hour,zone)
    value = (amount,count)
    return (key,value)

#### Tranform Value is reduce or get row value before and current value, 
#### for example old is v1 and current is v2 and return value like tuple data

In [ ]:
def tranformValue(oldValue,curValue):
    oldAmount,oldCount = oldValue
    curAmount,curCount = curValue

    totalAmount = oldAmount + curAmount
    totalCount = oldCount + curCount
    return (totalAmount,totalCount)

#### Unwrap is function for create new structure to dataframe

In [ ]:
def unWrap(row):
    return (row[0][1],row[0][0],row[1][0],row[1][1])

In [ ]:
rdd_schema = StructType([
    StructField("zone",LongType(),True),
    StructField("datetime",TimestampType(),True),
    StructField("amount",DoubleType(),True),
    StructField("total_transaction",LongType(),True),
])

#### Create new structure and convert to DataFrame

In [ ]:
rdd.filter(filter_outliers).map(prepareForGrouping).reduceByKey(tranformValue).map(unWrap).toDF(rdd_schema).show()

# Create Mechine Learning Model

In [28]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

#### Create new Model

In [155]:
def model_predict(df):
    y_pred = df.trip_distance * 5
    return y_pred

#### Iterate Model 

In [164]:
def apply_model_in_batch(partition):
    df = pd.DataFrame(partition,columns=columns)
    predictions = model_predict(df)
    df["prediction_duration"] = predictions
    return df.itertuples()

#### Create new structure data

In [165]:
df_predict = duration_rdd.mapPartitions(apply_model_in_batch).toDF().drop('Index')

#### Mechine Learning ready to use

In [177]:
df_predict.select("prediction_duration").show()

+-------------------+
|prediction_duration|
+-------------------+
|                6.0|
|               50.0|
| 28.900000000000002|
|               5.25|
|  7.800000000000001|
|              13.95|
|                4.8|
|                7.5|
|               25.8|
|               11.7|
|               4.25|
|               12.0|
|              50.05|
|                8.6|
|                5.2|
|                2.5|
|              100.7|
|               25.0|
| 106.44999999999999|
|                0.0|
+-------------------+
only showing top 20 rows



# Manipulate Partition and play with chunk data

#### Pick one partition

In [25]:
rowbe=duration_rdd.glom().collect()[0]

In [30]:
df = pd.DataFrame(rowbe,columns=columns)

#### Create chunk data

In [166]:
def infinite_loop():
    return df.itertuples()

#### Initiate Chunk Data

In [167]:
process_chunk = infinite_loop()

#### Call chunk data

In [176]:
data = next(process_chunk)
print(data)

Pandas(Index=8, VendorID=2, lpep_pickup_datetime=Timestamp('2019-01-23 16:34:55'), PULocationID=136, DOLocationID=42, trip_distance=5.16)


In [ ]:
for i in process_chunk:
    print(i)

    if i > 10:
        break

In [ ]:
df_predict.show()

In [ ]:
from multiprocessing.pool import ThreadPool
pool = ThreadPool(8)
pool.map(apply_model_in_batch,range(10))